# Inter-Annotator Agreement

In [1]:
import pandas as pd
from collections import Counter
import numpy as np
from nltk import agreement
from os import walk

In [2]:
def create_reduced_df(df):
    result = []
    for index, row in df.iterrows():
        WorkerId = row.WorkerId
        for i in range(1,41):
            text = row['Input.text' + str(i)]
            _id = row['Input.id' + str(i)]
            answer = row['Answer.classification' + str(i)]
            batch_id = row['Input.batch_id']
            result.append({'workerId': WorkerId, 'movie_id':_id, 'batch_id': batch_id, 'text': text, 'answer':answer, 'number' : i})
    df_result = pd.DataFrame(result)
    df_result['answer'].replace({'hatespeech': 2, 'normal': 0, 'offensive': 1}, inplace=True)
    return df_result

In [3]:
path = 'data/movies_raw'
_, _, movie_names = next(walk(path))

In [4]:
def load_all_movies(path, movie_names):
    
    df_list = []
    
    for movie in movie_names:
        df = pd.read_csv(path + '/' + movie)
        df = create_reduced_df(df)
        df['movie'] = movie.split('.')[0]
        df_list.append(df)
        
    result_df = pd.concat(df_list)
    result_df = result_df.dropna()
    
    return result_df

In [5]:
df = load_all_movies(path, movie_names)

In [6]:
df.head(1)

,workerId,movie_id,batch_id,text,answer,number,movie
0,AVNP1F3CADQRW,AmericanHistoryX(1998)_1,1566624979,Derek.,0,1,AmerricanHistoryX


In [7]:
len(df.workerId.unique())

13

In [8]:
movie_id_test = []
def get_annotated_movie(df):
    
    result = []
    
    batch_ids = df.batch_id.unique()
    for batch_id in batch_ids:
        movie_ids = df[(df.batch_id==batch_id)].movie_id.unique()
        for movie_id in movie_ids:
            annotation_single = df[(df.batch_id==batch_id) & (df.movie_id==movie_id)]
            
            if len(annotation_single)>0:
                movie_name = annotation_single.movie.values[0]
                majority_answer = Counter(annotation_single.answer.values)
                majority_answer_single = majority_answer.most_common()[0][0]
                
                text = annotation_single.text.values[0]
                
                if majority_answer.most_common()[0][1] == 1:
                    
#                     print('even')
#                     print(majority_answer)
#                     print(result)
                    majority_answer_single = 1
                    movie_id_test.append(movie_id)
                result_single = {'movie_id': movie_id, 'batch_id': batch_id, 'majority_answer': majority_answer_single, 'text': text, 'movie_name': movie_name}
                result.append(result_single)
            else:
                print('last batch')
    result_df = pd.DataFrame(result)
    return result_df
#             result_single = [list(x) for x in annotation_single[['workerId', 'movie_id', 'answer']].values]
#             if len(result_single) > 0:
#                 result.append()

In [9]:
result_df_movies_final = get_annotated_movie(df)

In [10]:
result_df_movies_final.to_csv('all_movies.csv')

In [11]:
result_df_movies_final

,movie_id,batch_id,majority_answer,text,movie_name
0,AmericanHistoryX(1998)_1,1566624979,0,Derek.,AmerricanHistoryX
1,AmericanHistoryX(1998)_2,1566624979,1,What the fuck are you thinking?,AmerricanHistoryX
2,AmericanHistoryX(1998)_3,1566624979,0,There's a black guy outside breaking into your...,AmerricanHistoryX
3,AmericanHistoryX(1998)_4,1566624979,0,How long has he been there?,AmerricanHistoryX
4,AmericanHistoryX(1998)_5,1566624979,0,I don't know.,AmerricanHistoryX
...,...,...,...,...,...
10683,TheWolfofWallStreet2013BluRay_3724,3859903933,0,Sell me this pen.,TheWolfofWallStreet
10684,TheWolfofWallStreet2013BluRay_3725,3859903933,0,"Well, it's a nice pen.",TheWolfofWallStreet
10685,TheWolfofWallStreet2013BluRay_3726_3727,3859903933,0,You can use the pen to write down thoughts fro...,TheWolfofWallStreet
10686,TheWolfofWallStreet2013BluRay_3728,3859903933,0,Sell me this pen.,TheWolfofWallStreet


In [12]:
def save_all_movies_for_training(result_df_movies_final):
    for name in result_df_movies_final.movie_name.unique():
        single_df = result_df_movies_final[result_df_movies_final.movie_name==name]
        single_df.to_csv(name + '_for_training' +  '.csv')

In [13]:
a = result_df_movies_final.majority_answer.value_counts().to_frame()
a['percentage'] = a['majority_answer'] / a.majority_answer.sum()

In [14]:
a.reset_index().rename(columns={'index': 'label'})

,label,majority_answer,percentage
0,0,9014,0.843376
1,1,1380,0.129117
2,2,294,0.027507


In [15]:
result_df_movies_final.groupby('movie_name').majority_answer.value_counts()

movie_name           majority_answer
AmerricanHistoryX    0                  1304
                     1                   207
                     2                    54
BlacKkKlansman       0                  1462
                     1                    97
                     2                    86
Django_Unchained     0                  1551
                     2                   117
                     1                    79
Pulp_Fiction         0                  1333
                     1                   265
                     2                    24
South_Park           0                   890
                     1                   145
                     2                    11
TheWolfofWallStreet  0                  2474
                     1                   587
                     2                     2
Name: majority_answer, dtype: int64

In [16]:
result_df_movies_final[(result_df_movies_final['movie_name']=='TheWolfofWallStreet') & (result_df_movies_final['majority_answer']==1)]

,movie_id,batch_id,majority_answer,text,movie_name
7631,TheWolfofWallStreet2013BluRay_7_8_9_10_11_12_13,3859903857,1,Stability. Integrity. Pride. One! Two! Three! ...,TheWolfofWallStreet
7643,TheWolfofWallStreet2013BluRay_33_34,3859903857,1,Yeah. She was the one with my cock in her mout...,TheWolfofWallStreet
7644,TheWolfofWallStreet2013BluRay_35,3859903857,1,So put your dick back in your pants.,TheWolfofWallStreet
7648,TheWolfofWallStreet2013BluRay_42,3859903857,1,"I fuck hookers maybe five, six times a week.",TheWolfofWallStreet
7672,TheWolfofWallStreet2013BluRay_77,3859903858,1,You can save the fucking spotted owl with money.,TheWolfofWallStreet
...,...,...,...,...,...
10657,TheWolfofWallStreet2013BluRay_3682,3859903932,1,It's fucking Chanel!,TheWolfofWallStreet
10660,TheWolfofWallStreet2013BluRay_3686_3687_3688,3859903932,1,"Like my pops, Mad Max, had said, ""The chickens...",TheWolfofWallStreet
10674,TheWolfofWallStreet2013BluRay_3709,3859903933,1,So listen. I've met some bad motherfuckers in ...,TheWolfofWallStreet
10676,TheWolfofWallStreet2013BluRay_3711,3859903933,1,"I'm telling you, some real bad motherfuckers.",TheWolfofWallStreet


In [17]:
result_df_movies_final.groupby('majority_answer').majority_answer.value_counts().plot(kind='bar')

<AxesSubplot:xlabel='majority_answer,majority_answer'>

result_df_movies_final.to_csv('AmericanHistoryX_majority_annotated.csv')

## calculate ties

In [18]:
df_ties = df.drop_duplicates('movie_id')

In [19]:
df_ties.describe

<bound method NDFrame.describe of             workerId                                 movie_id    batch_id  \
0      AVNP1F3CADQRW                 AmericanHistoryX(1998)_1  1566624979   
1      AVNP1F3CADQRW                 AmericanHistoryX(1998)_2  1566624979   
2      AVNP1F3CADQRW                 AmericanHistoryX(1998)_3  1566624979   
3      AVNP1F3CADQRW                 AmericanHistoryX(1998)_4  1566624979   
4      AVNP1F3CADQRW                 AmericanHistoryX(1998)_5  1566624979   
...              ...                                      ...         ...   
9138  A2H1RXUC8XNNIW       TheWolfofWallStreet2013BluRay_3724  3859903933   
9139  A2H1RXUC8XNNIW       TheWolfofWallStreet2013BluRay_3725  3859903933   
9140  A2H1RXUC8XNNIW  TheWolfofWallStreet2013BluRay_3726_3727  3859903933   
9141  A2H1RXUC8XNNIW       TheWolfofWallStreet2013BluRay_3728  3859903933   
9142  A2H1RXUC8XNNIW  TheWolfofWallStreet2013BluRay_3729_3730  3859903933   

                                         

### calculate ties.percentage

In [20]:
df_ties.shape[0]

10688

In [42]:
len(movie_id_test)

80

In [21]:
len(movie_id_test) / df_ties.shape[0]

0.0074850299401197605

In [22]:
result_df_movies_final[(result_df_movies_final.batch_id==1566625003)][['text', 'majority_answer']]

,text,majority_answer
280,Gun! He's got a gun!,0
281,Shit! Back up!,1
282,You make me fucking sick!,1
283,Put down the fucking gun!,1
284,Shoot the nigger!,2
285,"Shoot him! Do it, Seth! Shoot him!",0
286,"Traitor! Do it, Seth! Do it!",0
287,Back up! Move!,0
288,Back up! Back the fuck up!,1
289,Move! Move!,0


In [23]:
df_ties[df_ties.movie_id.isin(movie_id_test)]

,workerId,movie_id,batch_id,text,answer,number,movie
368,A182N7RLXGSCZG,AmericanHistoryX(1998)_382_383,1566624988,Good. I hate Tabitha Soren and her Zionist MTV...,1,9,AmerricanHistoryX
498,AVNP1F3CADQRW,AmericanHistoryX(1998)_182,1566624983,Minorities don't give two shits about this cou...,1,19,AmerricanHistoryX
871,A4D99Y82KOLC8,AmericanHistoryX(1998)_1029,1566625003,Just shit. Got totally out of control.,2,32,AmerricanHistoryX
1209,A4D99Y82KOLC8,AmericanHistoryX(1998)_1088,1566625005,Bad-ass peckerwood with an attitude.,0,10,AmerricanHistoryX
2299,A182N7RLXGSCZG,AmericanHistoryX(1998)_351,1566624987,When was the last time you were actually able ...,1,20,AmerricanHistoryX
...,...,...,...,...,...,...,...
1830,AVNP1F3CADQRW,TheWolfofWallStreet2013BluRay_772,3859903872,"You're a bunch of fucking sleazy salesmen, right?",2,31,TheWolfofWallStreet
3618,A182N7RLXGSCZG,TheWolfofWallStreet2013BluRay_1472,3859903887,You think this is 'cause you're a fag?,1,19,TheWolfofWallStreet
3620,A182N7RLXGSCZG,TheWolfofWallStreet2013BluRay_1474,3859903887,And I go on vacation with him and his boyfrien...,1,21,TheWolfofWallStreet
8064,A182N7RLXGSCZG,TheWolfofWallStreet2013BluRay_3293,3859903924,"I'm Swiss, I'm not communist.",1,25,TheWolfofWallStreet


In [24]:
df_ties[df_ties.movie_id.isin(movie_id_test)].groupby('workerId').answer.value_counts()

workerId        answer
A13WYZ8AXD6ODX  2          5
                1          3
                0          2
A182N7RLXGSCZG  2         18
                1          6
                0          2
A1L1S0IAPZB4MO  2          4
A2H1RXUC8XNNIW  0         10
                1          6
                2          2
A2MAGMY4JMDD3V  1          1
A4D99Y82KOLC8   0          5
                1          1
                2          1
AVNP1F3CADQRW   1          9
                2          5
Name: answer, dtype: int64

In [25]:
def get_percentage(answer, df):
    
    classifications = df[(df.answer==answer)].groupby('workerId').answer.value_counts().to_frame(name='total_classifications').reset_index('workerId')
    total_batches = df[(df.answer==answer)].drop_duplicates(subset=['batch_id', 'workerId']).groupby('workerId').batch_id.count().to_frame('total_batches').reset_index('workerId')
    
    merged_df = classifications.merge(total_batches, on='workerId')
    
    merged_df['number_dialogues'] = merged_df['total_batches'] * 40
    
    merged_df['percentage_as_' + str(answer)] = (merged_df['total_classifications'] / merged_df['number_dialogues'])
    
    merged_df.sort_values(by='number_dialogues' , ascending=False, inplace=True)
    return merged_df
    

In [26]:
get_percentage(0, df)

,workerId,total_classifications,total_batches,number_dialogues,percentage_as_0
5,A2H1RXUC8XNNIW,7116,213,8520,0.835211
1,A182N7RLXGSCZG,6451,198,7920,0.814520
11,A4D99Y82KOLC8,4743,138,5520,0.859239
12,AVNP1F3CADQRW,3570,113,4520,0.789823
0,A13WYZ8AXD6ODX,1380,40,1600,0.862500
6,A2MAGMY4JMDD3V,1272,39,1560,0.815385
10,A3E74I2YTGJCRF,634,22,880,0.720455
2,A1L1S0IAPZB4MO,128,16,640,0.200000
9,A32D8TKZV2RQ4Y,442,12,480,0.920833
8,A2V2ETRC6AO6GF,353,10,400,0.882500


In [27]:
get_percentage(1, df)

,workerId,total_classifications,total_batches,number_dialogues,percentage_as_1
5,A2H1RXUC8XNNIW,1100,192,7680,0.143229
1,A182N7RLXGSCZG,1071,181,7240,0.147928
11,A4D99Y82KOLC8,532,118,4720,0.112712
12,AVNP1F3CADQRW,726,109,4360,0.166514
6,A2MAGMY4JMDD3V,228,38,1520,0.150000
0,A13WYZ8AXD6ODX,112,36,1440,0.077778
10,A3E74I2YTGJCRF,206,22,880,0.234091
2,A1L1S0IAPZB4MO,367,15,600,0.611667
9,A32D8TKZV2RQ4Y,37,11,440,0.084091
8,A2V2ETRC6AO6GF,38,8,320,0.118750


In [28]:
get_percentage(2, df)

,workerId,total_classifications,total_batches,number_dialogues,percentage_as_2
1,A182N7RLXGSCZG,259,85,3400,0.076176
5,A2H1RXUC8XNNIW,187,77,3080,0.060714
12,AVNP1F3CADQRW,194,77,3080,0.062987
11,A4D99Y82KOLC8,145,59,2360,0.061441
0,A13WYZ8AXD6ODX,108,25,1000,0.108000
6,A2MAGMY4JMDD3V,60,19,760,0.078947
2,A1L1S0IAPZB4MO,110,15,600,0.183333
8,A2V2ETRC6AO6GF,9,6,240,0.037500
4,A2AA0GM76ENV5C,15,3,120,0.125000
10,A3E74I2YTGJCRF,5,3,120,0.041667


In [29]:
def get_fleiss_kappa(df, n, k):
    result = []
    annotation_all = []
    result_check = []
    df['answer'].replace({'hatespeech': 2, 'normal': 0, 'offensive': 1}, inplace=True)
    batch_ids = df.batch_id.unique()
    for batch_id in batch_ids:
        movie_ids = df[(df.batch_id==batch_id)].movie_id.unique()
        worker_ids = df[(df.batch_id==batch_id)].workerId.unique()
        
        annotation_singles = []
        for movie_id in movie_ids:
            annotation_single = df[(df.batch_id==batch_id) & (df.movie_id==movie_id)]
            f_k_single = [list(x) for x in annotation_single[['workerId', 'movie_id', 'answer']].values]
#             print(f_k_single)
#             break
            
            annotation_all.extend(f_k_single)
            annotation_singles.extend(f_k_single)
            result_check.append({'batch_id': str(batch_id), 'movie_id': movie_id, 'answer_1': f_k_single[0][2], 'answer_2': f_k_single[1][2],  'answer_3': f_k_single[2][2], 'worker_1': f_k_single[0][0], 'worker_2': f_k_single[1][0], 'worker_3': f_k_single[2][0]})
        # print(annotation_singles)
        ratingtask = agreement.AnnotationTask(annotation_singles)
        try:
            f_k= ratingtask.multi_kappa()
        except:
            f_k = 1
        result.append({'fleiss_kappa': f_k, 'batch_id': batch_id, 'worker_1': worker_ids[0], 'worker_2': worker_ids[1], 'worker_3': worker_ids[2]})
    result_df = pd.DataFrame(result)
    result_check_df = pd.DataFrame(result_check)
    return result_df, result_check_df, annotation_all

In [30]:
result_df, result_check_df, annotation_all = get_fleiss_kappa(df, n=3, k=3)

In [31]:
result_df

,fleiss_kappa,batch_id,worker_1,worker_2,worker_3
0,0.350649,1566624979,AVNP1F3CADQRW,A4D99Y82KOLC8,A182N7RLXGSCZG
1,0.485767,1566625000,AVNP1F3CADQRW,A4D99Y82KOLC8,A182N7RLXGSCZG
2,0.929577,1566625009,A2V2ETRC6AO6GF,A182N7RLXGSCZG,AVNP1F3CADQRW
3,0.468385,1566624988,A182N7RLXGSCZG,A4D99Y82KOLC8,AVNP1F3CADQRW
4,0.343750,1566624983,AVNP1F3CADQRW,A4D99Y82KOLC8,A182N7RLXGSCZG
...,...,...,...,...,...
266,0.916405,3859903929,A2MAGMY4JMDD3V,A182N7RLXGSCZG,A2H1RXUC8XNNIW
267,0.962512,3859903930,A182N7RLXGSCZG,A3E74I2YTGJCRF,A2H1RXUC8XNNIW
268,0.900000,3859903931,A2H1RXUC8XNNIW,A3E74I2YTGJCRF,A182N7RLXGSCZG
269,0.646465,3859903932,A182N7RLXGSCZG,A2H1RXUC8XNNIW,AVNP1F3CADQRW


In [32]:
result_df.fleiss_kappa.mean()

0.509893361378852

In [33]:
annotation_triples = [('A182N7RLXGSCZG', 'AmericanHistoryX(1998)_1405', 1), 
                       ('AVNP1F3CADQRW', 'AmericanHistoryX(1998)_1405', 1), 
                       ('A4D99Y82KOLC8', 'AmericanHistoryX(1998)_1405', 1)]

task = agreement.AnnotationTask(annotation_triples)

In [34]:
from sklearn.metrics import cohen_kappa_score
labeler1 = [1, 1, 0, 2]
labeler2 = [1, 1, 1, 1]
cohen_kappa_score(labeler1, labeler2)

0.0

In [35]:
result_check_df.head()

,batch_id,movie_id,answer_1,answer_2,answer_3,worker_1,worker_2,worker_3
0,1566624979,AmericanHistoryX(1998)_1,0,0,0,AVNP1F3CADQRW,A4D99Y82KOLC8,A182N7RLXGSCZG
1,1566624979,AmericanHistoryX(1998)_2,1,1,1,AVNP1F3CADQRW,A4D99Y82KOLC8,A182N7RLXGSCZG
2,1566624979,AmericanHistoryX(1998)_3,2,0,0,AVNP1F3CADQRW,A4D99Y82KOLC8,A182N7RLXGSCZG
3,1566624979,AmericanHistoryX(1998)_4,0,0,0,AVNP1F3CADQRW,A4D99Y82KOLC8,A182N7RLXGSCZG
4,1566624979,AmericanHistoryX(1998)_5,0,0,0,AVNP1F3CADQRW,A4D99Y82KOLC8,A182N7RLXGSCZG


In [46]:
result_check_df.agreement.value_counts(normalize=True)

1.000000    0.812874
0.666667    0.120977
0.333333    0.066149
Name: agreement, dtype: float64

In [60]:
disagreement_movoie_id = []
def fx(x):
    row = result_check_df[result_check_df.movie_id==x]
    answer_1 = row.answer_1.values[0]
    answer_2 = row.answer_2.values[0]
    answer_3 = row.answer_3.values[0]
    movie_id = row.movie_id.values[0]
    
    
    a = [answer_1, answer_2, answer_3]
    b = {}
    for i in a:
        b[i] = b.get(i, 0) + 1
    sorted_dict = dict(sorted(b.items()))
    number = list(sorted_dict.values())[0]
    
    if number == 3:
        result = 1  
    elif len(sorted_dict) == 3:
        result = 1/3
        disagreement_movoie_id.append(movie_id)
    else:
        result = 2/3
    return result

result_check_df['agreement'] = result_check_df.movie_id.apply(fx)

In [61]:
agreement_result = result_check_df.agreement.value_counts().to_frame()
agreement_result['percentage'] = agreement_result['agreement'] / agreement_result.agreement.sum()

In [62]:
agreement_result.reset_index().rename(columns={'index': 'percentage_they_do_agree'})

,percentage_they_do_agree,agreement,percentage
0,1.000000,8688,0.812874
1,0.666667,1920,0.179641
2,0.333333,80,0.007485


In [63]:
result_check_df[result_check_df.movie_id.isin(disagreement_movoie_id)]

,batch_id,movie_id,answer_1,answer_2,answer_3,worker_1,worker_2,worker_3,agreement
128,1566624988,AmericanHistoryX(1998)_382_383,2,1,3,A182N7RLXGSCZG,A4D99Y82KOLC8,AVNP1F3CADQRW,0.333333
178,1566624983,AmericanHistoryX(1998)_182,2,1,3,AVNP1F3CADQRW,A4D99Y82KOLC8,A182N7RLXGSCZG,0.333333
311,1566625003,AmericanHistoryX(1998)_1029,3,2,1,A4D99Y82KOLC8,AVNP1F3CADQRW,A182N7RLXGSCZG,0.333333
409,1566625005,AmericanHistoryX(1998)_1088,1,3,2,A4D99Y82KOLC8,AVNP1F3CADQRW,A182N7RLXGSCZG,0.333333
779,1566624987,AmericanHistoryX(1998)_351,2,1,3,A182N7RLXGSCZG,AVNP1F3CADQRW,A4D99Y82KOLC8,0.333333
...,...,...,...,...,...,...,...,...,...
8255,3859903872,TheWolfofWallStreet2013BluRay_772,3,1,2,AVNP1F3CADQRW,A182N7RLXGSCZG,A2H1RXUC8XNNIW,0.333333
8843,3859903887,TheWolfofWallStreet2013BluRay_1472,2,1,3,A182N7RLXGSCZG,A32D8TKZV2RQ4Y,A2H1RXUC8XNNIW,0.333333
8845,3859903887,TheWolfofWallStreet2013BluRay_1474,2,1,3,A182N7RLXGSCZG,A32D8TKZV2RQ4Y,A2H1RXUC8XNNIW,0.333333
10329,3859903924,TheWolfofWallStreet2013BluRay_3293,2,1,3,A182N7RLXGSCZG,A2H1RXUC8XNNIW,A3E74I2YTGJCRF,0.333333


In [39]:
def get_charts(df):
    df['answer_1'].replace({0: 1, 1: 2, 2: 3}, inplace=True)
    df['answer_2'].replace({0: 1, 1: 2, 2: 3}, inplace=True)
    df['answer_3'].replace({0: 1, 1: 2, 2: 3}, inplace=True)
    for batch_id in df.batch_id.unique():
        w_1, w_2, w_3 = df[df.batch_id==batch_id][['worker_1', 'worker_2', 'worker_3']].values[0] 
        title = 'batch_id: ' + str(batch_id) + '_' + 'w_1: ' + w_1 + '_' +  'w_2: ' + w_2 + '_' +  'w_3: ' + w_3
        df[df.batch_id==batch_id].plot(kind='bar', stacked=True, title=title)

In [40]:
get_charts(result_check_df)

C:\Users\Niklas\AppData\Roaming\Python\Python37\site-packages\pandas\plotting\_matplotlib\core.py:337: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)
